In [2]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import json

# Rewriting the Draft Algorithm with LA Operations

In [31]:
N_DRAFTERS = 4

M19_DECK_ARCHYTYPES = ("WU", "WB", "WR", "WG", "UB", "UR", "UG", "BR", "BG", "GR")
N_ARCHETYPES = len(M19_DECK_ARCHYTYPES)

In [32]:
CARD_VALUES_DICT = json.load(open('data/m19-custom-card-values-tuples-reduced.json'))['values']
CARD_NAMES = list(card_values)
N_CARDS = len(CARD_NAMES)

`player_preferences` is an `n_draters * n_arhetypes` array that tracks the player's preference for each archetype.  It is updated each time a player makes a draft pick.

In [34]:
player_prefercences = np.ones(shape=(N_DRAFTERS, N_ARCHETYPES))

`archetype_weights` is a `n_cards * n_archetypes` array containing the rating data for each card in each archetype.  This is what we would want a ML model to learn from actual draft data.

In [35]:
archetype_weights_df = pd.DataFrame(CARD_VALUES_DICT).T
archetype_weights_df.columns = M19_DECK_ARCHYTYPES

archetype_weights = archetype_weights_df.values

assert(archetype_weights.shape == (N_CARDS, N_ARCHETYPES))

`cards_in_pack` is an `n_drafters * n_cards` array that indicates which cards are in the players current pack.  Repeated cards are represented as counts.

In [36]:
from draftbot import Set

In [40]:
m19_reduced_data = json.load(open('data/m19-subset-reduced.json'))
m19_reduced_card_values = json.load(open('data/m19-custom-card-values-reduced.json'))
m19 = Set(cards=m19_reduced_data, card_values=m19_reduced_card_values)

In [52]:
packs = [m19.random_pack() for _ in range(N_DRAFTERS)]

cards_in_pack_df = pd.DataFrame(np.zeros(shape=(N_DRAFTERS, N_CARDS), dtype=int), 
                                columns=CARD_NAMES)

# This is probably inefficient...
for idx, pack in enumerate(packs):
    for card in pack:
        name = card['name']
        cards_in_pack_df.loc[cards_in_pack_df.index[idx], name] += 1

cards_in_pack = cards_in_pack_df.values

`card_is_in_pack` is an `n_drafters * n_cards` array that simply indicates if a card is in the pack.

In [56]:
card_is_in_pack = np.sign(cards_in_pack)

`card_is_in_pack_exploded` is a `n_drafters * n_cards * n_archetypes` array that simply copies the `card_is_in_pack` array once into each of the additional `n_archetypes` dimensions.

In [83]:
card_is_in_pack_exploded = np.repeat(card_is_in_pack[:, :, np.newaxis], N_ARCHETYPES, axis=2)

assert(card_is_in_pack_exploded.shape == (N_DRAFTERS, N_CARDS, N_ARCHETYPES))

In [85]:
par = np.einsum('dca,ca->dca', card_is_in_pack_exploded, archetype_weights)